## Generation of the GT for Training

In [1]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
export_ds = '/media/scratch/berlukas/nuscenes'
sem_clouds_filename = f"{export_ds}/clouds1_bw120_range_sem.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/clouds1_bw120_range_sem.npy
Shape of sem clouds is (11230, 2, 240, 240)


In [2]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes/kitti_processed'
sem_clouds_filename = f"{export_ds}/clouds-08.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/kitti_processed/clouds-08.npy
Shape of sem clouds is (4071, 3, 200, 200)


In [3]:
# only processing val
# n_clouds = sem_cloud_features.shape[0]
# print(f'we have in total {n_clouds}')
# n_train = n_clouds - 6053
# n_val = n_clouds - n_train
# print(f'we will keep {n_train} for training and {n_val} for validation.')
# sem_cloud_features = sem_cloud_features[n_train:, :, :, :]

we have in total 11467
we will keep 5414 for training and 6053 for validation.


In [3]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 1
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:, idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
n_unique_classes = len(np.unique(classes))
print(f"we have {n_unique_classes} classes in the dataset.")

  0%|          | 0/11230 [00:00<?, ?it/s]

shape of classes is (11230, 57600)
we have 33 classes in the dataset.


In [93]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 2
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (3, -1)).T    
    classes[i] = sem_cloud[:,idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
unique_classes = np.unique(classes)
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset.")
print(unique_classes)


shape of classes is (4071, 40000)
we have 29 classes in the dataset.
[ -1.   0.   1.  10.  11.  15.  18.  20.  30.  31.  32.  40.  44.  48.
  49.  50.  51.  52.  60.  70.  71.  72.  80.  81.  99. 252. 253. 254.
 259.]


## Iterate over the features and generate the GT

In [4]:
def map_nuscenes_label_old(label):
    noise = [0]
    vehicle = [1,2,3,4,5,6,7,8,9,10,11]
    living = [12,13,14,15,16,17,18,19]
    objects = [20,21,22,23]
    ground = [24,25,26,27]
    buildings = [28]
    vegetation = [29]
    other = [30]
    ego = [31]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 1
    elif label in living:
        return 2
    elif label in objects:
        return 3
    elif label in ground:
        return 4
    elif label in buildings:
        return 5
    elif label in vegetation:
        return 6
    elif label in other:
        return 7
    elif label in ego:
        return 8
    else:
        return 0

def map_nuscenes_label(label):
    noise = [0, 1, 5, 7, 8, 10, 11, 13, 19, 29, 31]
    living = [2,3,4,6]
    vehicle = [14,15,16,17,18,20,21,22,23]
    objects = [9,12]
    ground = [24,25,26,27]
    buildings = [28]
    vegetation = [30]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 1
    elif label in living:
        return 2
    elif label in objects:
        return 3
    elif label in ground:
        return 4
    elif label in buildings:
        return 5
    elif label in vegetation:
        return 6    
    else:
        return 0    
    
def map_kitti_to_nuscenes_label(label):
    noise = [0, 1, 11, 15, 16, 20,31,32, 44, 60, 71, 72, 80,81,99,253,255,256,259, 52]
    living = [30,254]
    vehicle = [10,13,18,252,257,258]
    objects = [51, 80, 81]
    ground = [40,48,49]
    buildings = [50]
    vegetation = [70, 71]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 1
    elif label in living:
        return 2
    elif label in objects:
        return 3
    elif label in ground:
        return 4
    elif label in buildings:
        return 5
    elif label in vegetation:
        return 6    
    else:
        if label >= 0:
            print(f'WARNING CLASS LABEL {label} not handled!!!!')
        return 0    
    
def map_nuscenes_label_16(label):                       
    noise = [1, 5, 6, 8, 10, 11, 13, 19, 20, 0, 29, 31]
    barrier = [9]
    bicycle = [14]
    bus = [15, 16]
    car = [17]
    construction_vehicle = [18]
    motorcycle = [21]
    pedestrian = [2, 3, 4, 6]
    traffic_cone = [12]
    trailer = [22]
    truck = [23]
    driveable_surface = [24]
    other_flat = [25]
    sidewalk = [26]
    terrain = [27]
    manmade = [28]
    vegetation = [30]
        
    if label in noise:
        return 0
    elif label in barrier:
        return 1
    elif label in bicycle:
        return 2
    elif label in bus:
        return 3
    elif label in car:
        return 4
    elif label in construction_vehicle:
        return 5
    elif label in motorcycle:
        return 6
    elif label in pedestrian:
        return 7
    elif label in traffic_cone:
        return 8
    elif label in trailer:
        return 9
    elif label in truck:
        return 10
    elif label in driveable_surface:
        return 11
    elif label in other_flat:
        return 12
    elif label in sidewalk:
        return 13
    elif label in terrain:
        return 14
    elif label in manmade:
        return 15
    elif label in vegetation:
        return 16    
    else:
        return 0

In [7]:
n_features = sem_cloud_features.shape[0]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]
idx_sem = 1

print(f'Iterating for {n_features} features and data of size: ({n_rows}/{n_cols})')
for feature in tqdm(range(n_features)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = map_nuscenes_label_old(cur_class)
            mapped_class = map_nuscenes_label(cur_class)            
            sem_cloud_features[feature, idx_sem, i, j] = mapped_class

Iterating for 11230 features and data of size: (240/240)


  0%|          | 0/11230 [00:00<?, ?it/s]

In [5]:
# sem_cloud_features = sem_cloud_features[0:10,:,:,:]
idx_sem = 2
n_data = sem_cloud_features.shape[0]
n_features = sem_cloud_features.shape[1]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]

idx_intensity = 1;
for feature in tqdm(range(n_data)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])            
            mapped_class = map_kitti_to_nuscenes_label(cur_class)
            
            cur_intensity = sem_cloud_features[feature, idx_intensity, i, j]
#             if cur_intensity > 0:
#                 cur_intensity = ((cur_intensity ) / 0.16)
#                 cur_intensity = cur_intensity * 255 
            
#             mapped_class = map_nuscenes_label(cur_class)
#             mapped_class = map_nuscenes_label_old(cur_class)
            sem_cloud_features[feature, idx_sem, i, j] = mapped_class
#             sem_cloud_features[feature, idx_intensity, i, j] = int(cur_intensity)
#             if cur_intensity > 1:
#                 print(sem_cloud_features[feature, idx_intensity, i, j])

In [8]:
filename = 'sem_clouds_01_bw120'
sem_clouds_gt_filename = f'{export_ds}/{filename}.npy'
sem_clouds_gt_tiny_filename = f'{export_ds}/{filename}_tiny.npy'
np.save(sem_clouds_gt_filename, sem_cloud_features)

n_process = 400
np.save(sem_clouds_gt_tiny_filename, sem_cloud_features[0:n_process, :, :, :])

print(f"Wrote features to {sem_clouds_gt_filename}")
print(f"Wrote tiny features to {sem_clouds_gt_tiny_filename}")

Wrote features to /media/scratch/berlukas/nuscenes/sem_clouds_01_bw120.npy
Wrote tiny features to /media/scratch/berlukas/nuscenes/sem_clouds_01_bw120_tiny.npy


In [8]:
# Remove a few samples for validation
n_clouds = sem_cloud_features.shape[0]
print(f'we have in total {n_clouds}')
n_val_clouds = 6053
n_train = n_clouds - 400
n_val = n_clouds - n_train
print(f'we will keep {n_train} for training and {n_val} for validation.')

print(f'init shape is {sem_cloud_features.shape}')
sem_train = sem_cloud_features[:n_train, :, :, :]
sem_val = sem_cloud_features[n_train:, :, :, :]
print(f'training shape is {sem_train.shape}, validation shape is {sem_val.shape}')

np.save(f'{export_ds}/sem_clouds3.npy', sem_train)
np.save(f'{export_ds}/sem_clouds_val_200.npy', sem_val)

we have in total 11467
we will keep 11067 for training and 400 for validation.
init shape is (11467, 3, 200, 200)
training shape is (11067, 3, 200, 200), validation shape is (400, 3, 200, 200)


In [9]:
sem_val_tiny = sem_val[:500, :, :, :]
np.save(f'{export_ds}/sem_clouds_val_tiny.npy', sem_val_tiny)